In [ ]:
import shutil
from autumn.projects.sm_covid2.common_school.runner_tools import INCLUDED_COUNTRIES
import pandas as pd
from pathlib import Path
from autumn.projects.sm_covid2.common_school.output_plots.multicountry import plot_multic_relative_outputs, plot_relative_map
from matplotlib import pyplot as plt 

full_iso3_list = list(INCLUDED_COUNTRIES['all'].keys())

In [18]:
# analysis_folder = Path.cwd() / "31747883_full_analysis_26Sep2023_main"
# analysis_folder = Path.cwd() /"31902886_full_analysis_05Oct2023_increased_hh_contacts"

analysis_folder = Path.cwd() /"31915437_full_analysis_05Oct2023_no_google_mobility"


### Move files to common folder

In [19]:
common_dir = analysis_folder / "COMMON_DIRECTORY"
common_dir.mkdir(exist_ok=True)


In [ ]:

to_move = {
    "r_hats": "mc_outputs/r_hats.jpg",
    "mcmc_traces_postburnin": "mc_outputs/mc_traces_postburnin.jpg",
    "optimal_fits": "optimal_fits.png",
}

for folder_name, file in to_move.items():
    folder = common_dir / folder_name
    folder.mkdir(exist_ok=True)

    for iso3 in full_iso3_list:
        src = analysis_folder / iso3 / file 
        if Path.exists(src):
            dst = folder / f"{src.stem}_{iso3}{src.suffix}" 
            shutil.copy(src, dst)

### Combine PDF files

In [20]:
from pypdf import PdfMerger

pdf_paths = []
for iso3 in full_iso3_list:
    pdf_path = analysis_folder / iso3 / "tiling.pdf"
    if Path.exists(pdf_path):
        pdf_paths.append(pdf_path)

merger = PdfMerger()

for pdf in pdf_paths:
    merger.append(pdf)

merger.write(common_dir / "multi_tiling.pdf")
merger.close()

### Make multi-country figures

In [21]:
FILE_FORMATS = ["png", "pdf"]

In [22]:
diff_quantiles_dfs = {}
for iso3 in full_iso3_list:
    diff_quantiles_path = analysis_folder / iso3 / "diff_quantiles_df.parquet"
    if Path.exists(diff_quantiles_path):
        diff_quantiles_dfs[iso3] = pd.read_parquet(diff_quantiles_path)

In [23]:
fig = plot_multic_relative_outputs(diff_quantiles_dfs)
plt.close()
for file_format in FILE_FORMATS:
    fig.savefig(common_dir / f"relative_outputs.{file_format}")

In [24]:
(common_dir / "maps").mkdir(exist_ok=True)
for output in ["cases_averted_relative", "delta_hospital_peak_relative", "deaths_averted_relative"]:
    fig = plot_relative_map(diff_quantiles_dfs, output)
    plt.close()
    for file_format in FILE_FORMATS:
        fig.write_image(common_dir / "maps" / f"map_{output}.{file_format}")